In [81]:
import pandas as pd
import numpy as np
import pygeohash as pgh

import seaborn as sns

In [12]:
df = pd.read_csv('./worked_data.csv')

In [13]:
df.head()

,Unnamed: 0,product_id,name,unit_price,geohash,primary_cuisine,en_name
0,1,ec33db14a2c5,alcachofas empinzadas,0.304,u6scd,spanish,alcachofas empinzadas
1,2,fa04d9e5b964,coca-cola 2l,0.240,u6scq,pizza,coca-cola 2l
2,3,9f502de9373e,"5 lax, 5 räkor, 5 avokado",0.820,u6sck,sushi,"5 salmon, 5 shrimp, 5 avocados"
3,5,35ac685bcc43,52 femtiotvå,0.872,u6sct,thai,52 fifty-two
4,6,ea2ce1a7db0f,sprite 33 cl,0.152,u6sct,thai,sprite 33 cl


In [14]:
lat = []
long = []
for index, row in df.iterrows(): 
    x, y = pgh.decode(row['geohash'])
    lat.append(x)
    long.append(y)
df['latitude'] = lat
df['longitude'] = long

In [15]:
df = df.drop(columns = ['Unnamed: 0', 'name', 'geohash'])

In [16]:
df.head()

,product_id,unit_price,primary_cuisine,en_name,latitude,longitude
0,ec33db14a2c5,0.304,spanish,alcachofas empinzadas,59.3,18.0
1,fa04d9e5b964,0.240,pizza,coca-cola 2l,59.3,18.2
2,9f502de9373e,0.820,sushi,"5 salmon, 5 shrimp, 5 avocados",59.3,18.1
3,35ac685bcc43,0.872,thai,52 fifty-two,59.3,18.2
4,ea2ce1a7db0f,0.152,thai,sprite 33 cl,59.3,18.2


## Эмбединг слов

In [17]:
!pip install gensim


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [48]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    min_count=1,
    window=1,
    negative=10,
    vector_size = 10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [49]:
df.isna().value_counts()

product_id  unit_price  primary_cuisine  en_name  latitude  longitude
False       False       False            False    False     False        96578
dtype: int64

In [50]:
df = df.fillna('')

In [51]:
data = [df['primary_cuisine'].to_list() + df['en_name'].to_list()]
w2v_model.build_vocab(data)

In [52]:
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(300000, 5794680)

In [53]:
w2v_model.wv['pizza']

array([ 1.1345505 , -1.3526487 ,  2.7670715 , -0.3965687 , -0.10437916,
       -1.8213797 ,  8.1703825 ,  5.3759084 , -6.5864162 , -0.93779707],
      dtype=float32)

In [56]:
new_df = df.copy()
new_df = new_df.drop(columns = ['primary_cuisine', 'en_name'])

for index, row in df.iterrows(): 
    primary_cuisine = w2v_model.wv[row['primary_cuisine']]
    en_name = w2v_model.wv[str(row['en_name'])]
    i = 0
    for n in primary_cuisine:
        new_df.at[index, f'primary_cuisine_{i}'] = n
        i += 1
    i = 0
    for n in en_name:
        new_df.at[index, f'en_name_{i}'] = n
        i += 1

In [57]:
new_df.head()

,product_id,unit_price,latitude,longitude,primary_cuisine_0,primary_cuisine_1,primary_cuisine_2,primary_cuisine_3,primary_cuisine_4,primary_cuisine_5,...,en_name_0,en_name_1,en_name_2,en_name_3,en_name_4,en_name_5,en_name_6,en_name_7,en_name_8,en_name_9
0,ec33db14a2c5,0.304,59.3,18.0,0.757551,-1.042341,2.151450,-0.223921,-0.246398,-1.274192,...,0.124920,-0.107810,0.327576,-0.098199,0.027440,-0.137183,0.658900,0.392239,-0.452834,-0.115296
1,fa04d9e5b964,0.240,59.3,18.2,1.134550,-1.352649,2.767071,-0.396569,-0.104379,-1.821380,...,0.092733,-0.076130,0.437493,-0.296371,0.219356,-0.086068,1.059721,0.676011,-0.875360,-0.058168
2,9f502de9373e,0.820,59.3,18.1,0.882450,-1.230438,2.528480,-0.481319,-0.097049,-1.553277,...,0.252368,-0.100887,0.385945,-0.244434,0.184637,-0.164916,0.867720,0.454744,-0.801865,-0.023821
3,35ac685bcc43,0.872,59.3,18.2,0.957896,-1.058989,2.559645,-0.416770,-0.043126,-1.555766,...,0.170897,-0.110023,0.433800,-0.320116,0.070227,-0.219308,0.893524,0.567447,-0.880274,-0.117358
4,ea2ce1a7db0f,0.152,59.3,18.2,0.957896,-1.058989,2.559645,-0.416770,-0.043126,-1.555766,...,0.232580,-0.324344,0.621244,-0.457554,0.217313,-0.372510,1.596732,0.924262,-1.355715,-0.066245


In [62]:
from sklearn.model_selection import train_test_split

In [63]:
X_train, X_test = train_test_split(new_df, test_size=0.33, random_state=42)

## Кластеризация

In [59]:
!pip install hdbscan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 9.7 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp39-cp39-macosx_11_0_arm64.whl size=620720 sha256=a2b887e36b2376a58d7951f7e48217bd2fa6f43774786f7ebf28496a802b164a
  Stored in directory: /Users/tainella/Library/Caches/pip/wheels/28/5e/ed/5989da4cc423a222a47cbb4fde5d6c0eff4590d922e45f233c
Successfully built hdbscan

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [60]:
import hdbscan

In [85]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=20, gen_min_span_tree=True)
clusterer.fit(X_train.drop(columns=['product_id']))

HDBSCAN(gen_min_span_tree=True, min_cluster_size=20)

In [86]:
np.unique(clusterer.labels_)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 17

In [89]:
clusterer.condensed_tree_.to_pandas().head()

,parent,child,lambda_val,child_size
0,64707,64708,0.240351,436
1,64707,64709,0.240351,64271
2,64708,41099,0.258756,1
3,64709,24075,0.730359,1
4,64708,48114,0.261541,1
